In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import mplfinance as mpf
import os
from pypfopt.efficient_frontier import EfficientFrontier
import pypfopt.base_optimizer
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import sample_cov

In [3]:
data = pd.read_csv('mag7.csv', header=[0, 1], index_col=0, parse_dates=True)
data.tail(5)

AAPL                                                  \
             Adj Close       Close        High         Low        Open   
Date                                                                     
2023-12-22  193.091385  193.600006  195.410004  192.970001  195.179993   
2023-12-26  192.542816  193.050003  193.889999  192.830002  193.610001   
2023-12-27  192.642548  193.149994  193.500000  191.089996  192.490005   
2023-12-28  193.071426  193.580002  194.660004  193.169998  194.139999   
2023-12-29  192.024185  192.529999  194.399994  191.729996  193.899994   

                            AMZN                                      ...  \
              Volume   Adj Close       Close        High         Low  ...   
Date                                                                  ...   
2023-12-22  37122800  153.419998  153.419998  154.350006  152.710007  ...   
2023-12-26  28919300  153.410004  153.410004  153.979996  153.029999  ...   
2023-12-27  48087700  153.339996  153.339996  154.779999  153.119995  ...   
2023-12-28  34049900  153.380005  153.380005  154.080002  152.949997  ...   
2023-12-29  42628800  151.940002  151.940002  153.889999  151.029999  ...   

                 NVDA                                         TSLA  \
                 High        Low       Open     Volume   Adj Close   
Date                                                                 
2023-12-22  49.382999  48.466999  49.195000  252507000  252.539993   
2023-12-26  49.599998  48.959999  48.967999  244200000  256.609985   
2023-12-27  49.680000  49.084999  49.511002  233648000  261.440002   
2023-12-28  49.883999  49.411999  49.643002  246587000  253.179993   
2023-12-29  49.997002  48.750999  49.813000  389293000  248.479996   

                                                                       
                 Close        High         Low        Open     Volume  
Date                                                                   
2023-12-22  252.539993  258.220001  251.369995  256.760010   93249800  
2023-12-26  256.609985  257.970001  252.910004  254.490005   86892400  
2023-12-27  261.440002  263.339996  257.519989  258.350006  106494400  
2023-12-28  253.179993  265.130005  252.710007  263.660004  113619900  
2023-12-29  248.479996  255.190002  247.429993  255.100006  100615300  

[5 rows x 42 columns]

## Trying with AAPL

In [4]:
aapl = data['AAPL'][['Close', 'High', 'Low']]

aapl['rolling_mean'] = aapl['Close'].rolling(window=20).mean()
aapl['rolling_std'] = aapl['Close'].rolling(window=20).std()
aapl['A1'] = aapl['rolling_mean'] + (aapl['rolling_std'] * 2)
# aapl['A1'] = aapl['A1'].shift(1)
aapl['A2'] = aapl['rolling_mean'] - (aapl['rolling_std'] * 2)
# aapl['A2'] = aapl['A2'].shift(1)
aapl['B1'] = aapl['rolling_mean'] + (aapl['rolling_std'] * 1)
# aapl['B1'] = aapl['B1'].shift(1)
aapl['B2'] = aapl['rolling_mean'] - (aapl['rolling_std'] * 1)
# aapl['B2'] = aapl['B2'].shift(1)

aapl['26MA'] = aapl['Close'].ewm(span=26, adjust=False).mean()
# aapl['26MA'] = aapl['26MA'].shift(1)
aapl['12MA'] = aapl['Close'].ewm(span=12, adjust=False).mean()
# aapl['12MA'] = aapl['12MA'].shift(1)
aapl['MACD'] = aapl['12MA'] - aapl['26MA']
# aapl['MACD'] = aapl['MACD'].shift(1)
aapl['Signal'] = aapl['MACD'].ewm(span=9, adjust=False).mean()

aapl

,Close,High,Low,rolling_mean,rolling_std,A1,A2,B1,B2,26MA,12MA,MACD,Signal
Date,,,,,,,,,,,,,
2013-01-02,19.608213,19.821428,19.343929,NaN,NaN,NaN,NaN,NaN,NaN,19.608213,19.608213,0.000000,0.000000
2013-01-03,19.360714,19.631071,19.321428,NaN,NaN,NaN,NaN,NaN,NaN,19.589880,19.570137,-0.019744,-0.003949
2013-01-04,18.821428,19.236786,18.779642,NaN,NaN,NaN,NaN,NaN,NaN,19.532958,19.454951,-0.078007,-0.018760
2013-01-07,18.710714,18.903570,18.400000,NaN,NaN,NaN,NaN,NaN,NaN,19.472051,19.340453,-0.131598,-0.041328
2013-01-08,18.761070,18.996071,18.616072,NaN,NaN,NaN,NaN,NaN,NaN,19.419386,19.251317,-0.168069,-0.066676
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,193.600006,195.410004,192.970001,193.668501,2.911017,199.490535,187.846466,196.579518,190.757483,192.102573,194.738889,2.636317,3.236689
2023-12-26,193.050003,193.889999,192.830002,193.831501,2.770282,199.372064,188.290938,196.601783,191.061219,192.172753,194.479061,2.306308,3.050613
2023-12-27,193.149994,193.500000,191.089996,193.969001,2.656926,199.282852,188.655150,196.625927,191.312075,192.245141,194.274589,2.029448,2.846380


## Generating signal dataframe

In [5]:
signal_df = pd.DataFrame(index=aapl.index)

signal_df['position'] = np.where(((aapl['MACD'] > aapl['Signal']) & (aapl['High'] > aapl['B1'])), 1, 0)
signal_df['position'] = np.where(((aapl['MACD'] < aapl['Signal']) & (aapl['Low'] < aapl['B2'])), -1, signal_df['position'])
signal_df['position'] = signal_df['position'].shift(1)
signal_df['position'] = signal_df['position'].fillna(0)
signal_df

,position
Date,
2013-01-02,0.0
2013-01-03,0.0
2013-01-04,0.0
2013-01-07,0.0
2013-01-08,0.0
...,...
2023-12-22,0.0
2023-12-26,0.0
2023-12-27,0.0


## Generating returns

In [6]:
returns_df = aapl[['Close']].pct_change()
returns_df['returns'] = returns_df['Close'] * signal_df['position']
returns_df

,Close,returns
Date,,
2013-01-02,NaN,NaN
2013-01-03,-0.012622,-0.000000
2013-01-04,-0.027855,-0.000000
2013-01-07,-0.005882,-0.000000
2013-01-08,0.002691,0.000000
...,...,...
2023-12-22,-0.005547,-0.000000
2023-12-26,-0.002841,-0.000000
2023-12-27,0.000518,0.000000


## Strategy returns

In [7]:
(returns_df['returns'] +1).cumprod()

Date
2013-01-02         NaN
2013-01-03    1.000000
2013-01-04    1.000000
2013-01-07    1.000000
2013-01-08    1.000000
                ...   
2023-12-22    5.474540
2023-12-26    5.474540
2023-12-27    5.474540
2023-12-28    5.462352
2023-12-29    5.462352
Name: returns, Length: 2768, dtype: float64

## Buy and hold

In [8]:
(returns_df['Close']+1).cumprod()

Date
2013-01-02         NaN
2013-01-03    0.987378
2013-01-04    0.959875
2013-01-07    0.954228
2013-01-08    0.956797
                ...   
2023-12-22    9.873414
2023-12-26    9.845364
2023-12-27    9.850464
2023-12-28    9.872394
2023-12-29    9.818845
Name: Close, Length: 2768, dtype: float64

Based on comparison between strategy and buy and hold during the period, buy and hold seems to perform much better

## Writing OOP for trading strategy

In [15]:
def generate_indicator(ticker_data: pd.DataFrame,) -> pd.DataFrame:
    """
    Return a DataFrame with the DBB and MACD indicators based on the stock's close, high and low
    """
    indicator_df = ticker_data.copy()
    indicator_df['rolling_mean'] = indicator_df['Close'].rolling(window=20).mean()
    indicator_df['rolling_std'] = indicator_df['Close'].rolling(window=20).std()
    indicator_df['A1'] = indicator_df['rolling_mean'] + (indicator_df['rolling_std'] * 2)
    indicator_df['A2'] = indicator_df['rolling_mean'] - (indicator_df['rolling_std'] * 2)
    indicator_df['B1'] = indicator_df['rolling_mean'] + (indicator_df['rolling_std'] * 1)
    indicator_df['B2'] = indicator_df['rolling_mean'] - (indicator_df['rolling_std'] * 1)

    indicator_df['26MA'] = indicator_df['Close'].ewm(span=26, adjust=False).mean()
    indicator_df['12MA'] = indicator_df['Close'].ewm(span=12, adjust=False).mean()
    indicator_df['MACD'] = indicator_df['12MA'] - indicator_df['26MA']
    indicator_df['Signal'] = indicator_df['MACD'].ewm(span=9, adjust=False).mean()

    return indicator_df

def generate_signal(ticker_data: pd.DataFrame) -> pd.DataFrame:
    """
    Return a DataFrame with either 1, -1 or 0 based on trading rules
    """
    signal_df = pd.DataFrame(index=ticker_data.index)

    signal_df['position'] = np.where(((ticker_data['MACD'] > ticker_data['Signal']) & (ticker_data['High'] > ticker_data['B1'])), 1, 0)
    signal_df['position'] = np.where(((ticker_data['MACD'] < ticker_data['Signal']) & (ticker_data['Low'] < ticker_data['B2'])), -1, signal_df['position'])
    signal_df['position'] = signal_df['position'].shift(1)
    signal_df['position'] = signal_df['position'].fillna(0)
    return signal_df

class DBB_MACD_Strategy:
    def __init__(self, 
                 ticker_data: pd.DataFrame,
                 indicator_df: pd.DataFrame,
                 signal_df: pd.DataFrame):
        self.ticker_data = ticker_data
        self.indicator_df = indicator_df
        self.signal_df = signal_df

    def generate_returns(self):
        stock_pct_change = self.ticker_data['Close'].pct_change()
        returns = stock_pct_change * self.signal_df['position']
        self.strategy_returns = (returns + 1).cumprod()
        return self.strategy_returns
    
    def sharpe_ratio(self):
        self.strategy_returns = self.generate_returns()
        return self.strategy_returns.pct_change().mean() / self.strategy_returns.pct_change().std()

    def max_drawdown(self):
        self.strategy_returns = self.generate_returns()
        return (self.strategy_returns / self.strategy_returns.cummax() - 1).min()

In [16]:
indicator_df = generate_indicator(aapl)
signal_df = generate_signal(indicator_df)
strategy = DBB_MACD_Strategy(aapl, indicator_df, signal_df)
# strategy_returns = strategy.generate_returns()
# strategy.sharpe_ratio()
strategy.max_drawdown()

-0.2577149600261087